# Credit Risk Resampling Techniques

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

In [8]:
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

## Read the CSV into DataFrame

In [9]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


## Split the Data into Training and Testing

In [10]:
# Create our features
X = df.copy()
X.drop(columns = ["loan_status", "homeowner"], axis= 1, inplace = True)

# Create our target
y = df["loan_status"]

In [11]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [12]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [13]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    random_state = 1,
                                                    stratify =y
                                                   )
X_train.shape

(58152, 7)

## Data Pre-Processing

In [14]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [15]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [16]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Part 1: Simple Logistic Regression

In [48]:
# Counter
Counter(y_train)

Counter({'low_risk': 56277, 'high_risk': 1875})

In [54]:
# Train the Logistic Regression model
lr_model = LogisticRegression(solver='lbfgs', random_state=1)
lr_model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [50]:
# Calculated the balanced accuracy score
y_pred_lr = lr_model.predict(X_test)
lr_score = balanced_accuracy_score(y_test, y_pred_lr)
lr_score

0.9543211898288821

In [51]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_lr)

array([[  571,    54],
       [   93, 18666]])

In [52]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_lr))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



## Part 2: Naive Random Oversampling

In [22]:
# Counter
Counter(y_train)

Counter({'low_risk': 56277, 'high_risk': 1875})

In [23]:
# Resample the training data with the RandomOverSampler
# View the count of target classes with Counter
ros_model = RandomOverSampler(random_state = 1)
X_resampled, y_resampled = ros_model.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [24]:
# Train the Logistic Regression model using the resampled data
ros_model = LogisticRegression(solver = 'lbfgs', random_state = 1)
ros_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [55]:
# Calculated the balanced accuracy score
y_pred_ros = ros_model.predict(X_resampled)

ros_score = balanced_accuracy_score(y_resampled, y_pred_ros)
ros_score

0.9989887103298642

In [56]:
# Display the confusion matrix
confusion_matrix(y_resampled, y_pred_ros)

array([[54276,   110],
       [    0, 55907]])

In [27]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_resampled, y_pred_ros))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.99      0.99      0.99      0.99      0.99      0.99     56277
   low_risk       0.99      0.99      0.99      0.99      0.99      0.99     56277

avg / total       0.99      0.99      0.99      0.99      0.99      0.99    112554



## Part 3: SMOTE Oversampling

In [28]:
# Counter
Counter(y_train)

Counter({'low_risk': 56277, 'high_risk': 1875})

In [29]:
# Resample the training data with SMOTE
X_resampled, y_resampled = SMOTE(random_state = 1, sampling_strategy = 1.0).fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [30]:
# Train the Logistic Regression model using the resampled data
SMOTE_model = LogisticRegression(solver = 'lbfgs', random_state = 1)
SMOTE_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [31]:
# Calculated the balanced accuracy score
y_pred_SMOTE = SMOTE_model.predict(X_resampled)
SMOTE_score = balanced_accuracy_score(y_resampled, y_pred_SMOTE)
SMOTE_score

0.9946159176928409

In [32]:
# Display the confusion matrix
confusion_matrix(y_resampled, y_pred_SMOTE)

array([[55989,   288],
       [  318, 55959]])

In [33]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_resampled, y_pred_SMOTE))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.99      0.99      0.99      0.99      0.99      0.99     56277
   low_risk       0.99      0.99      0.99      0.99      0.99      0.99     56277

avg / total       0.99      0.99      0.99      0.99      0.99      0.99    112554



# Part 4: Cluster Centroids Undersampling

In [34]:
# Resample the data using the ClusterCentroids resampler
cc_model = ClusterCentroids(random_state = 1)
X_resampled, y_resampled = cc_model.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [35]:
# Train the Logistic Regression model using the resampled data
cc_model = LogisticRegression(solver = 'lbfgs', random_state = 1)
cc_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [57]:
# Calculate the balanced accuracy score
y_pred_cc = cc_model.predict(X_resampled)
cc_score = balanced_accuracy_score(y_resampled, y_pred_cc)
cc_score

0.9911558121575406

In [58]:
# Display the confusion matrix
confusion_matrix(y_resampled, y_pred_cc)

array([[53424,   962],
       [    0, 55907]])

In [38]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_resampled, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.90      0.98      0.89      0.94      0.94      0.88      1875
   low_risk       0.98      0.89      0.98      0.93      0.94      0.87      1875

avg / total       0.94      0.94      0.94      0.94      0.94      0.88      3750



# Part 5: SMOTEENN Combination (Over and Under) Sampling

In [39]:
# Resample the training data with SMOTEENN
SMOTEENN_model = SMOTEENN(random_state = 1)
X_resampled, y_resampled = SMOTEENN_model.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 54386, 'low_risk': 55907})

In [40]:
# Train the Logistic Regression model using the resampled data
SMOTEENN_model = LogisticRegression(solver = 'lbfgs', random_state = 1)
SMOTEENN_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [59]:
# Calculate the balanced accuracy score
y_pred_SMOTEENN = SMOTEENN_model.predict(X_resampled)
SMOTEENN_score = balanced_accuracy_score(y_resampled, y_pred_SMOTEENN)
SMOTEENN_score

0.9991358070091567

In [60]:
# Display the confusion matrix
confusion_matrix(y_resampled, y_pred_SMOTEENN)

array([[54292,    94],
       [    0, 55907]])

In [43]:
# Print the imbalanced classification re
print(classification_report_imbalanced(y_resampled, y_pred_SMOTEENN))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       1.00      1.00      1.00      1.00      1.00      1.00     54386
   low_risk       1.00      1.00      1.00      1.00      1.00      1.00     55907

avg / total       1.00      1.00      1.00      1.00      1.00      1.00    110293



# Final Questions

1. Which model had the best balanced accuracy score?

   Smoteenn Model.

2. Which model had the best recall score?

    Smoteenn Model.

3. Which model had the best geometric mean score?

    Smoteenn Model.


In [53]:
# Create DataFrame for accuracy scores
scores = {
    "scores": {
    "Logistic Regression Score": lr_score, 
    "Random Oversampling Score": ros_score,
    "SMOTE Oversampling Score": SMOTE_score,
    "Cluster Centroids Undersampling Score": cc_score,
    "SMOTEENN Score": SMOTEENN_score
    }
}
scores = pd.DataFrame(scores)

# View dataframe
scores

,scores
Cluster Centroids Undersampling Score,0.936533
Logistic Regression Score,0.954321
Random Oversampling Score,0.994261
SMOTE Oversampling Score,0.994616
SMOTEENN Score,0.999136


In [45]:
# Get index for Max score
scores.index[scores["scores"] == scores["scores"].max()]

Index(['SMOTEENN Score'], dtype='object')